<a href="https://colab.research.google.com/github/venkatchintha/Project0/blob/master/Monk_to_COCO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [155]:
from google.colab import drive
drive.mount('/content/drive',force_remount= True)

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Kaggle/nexet

[Errno 2] No such file or directory: '/content/drive/My Drive/Kaggle/nexet'
/content


In [160]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: could not create work tree dir 'Monk_Object_Detection': No such file or directory


In [69]:
! cd Monk_Object_Detection/3_mxrcnn/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install

In [70]:
import os
import numpy as np 
import cv2
import dicttoxml
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString
from tqdm import tqdm
import shutil
import json
import pandas as pd

In [147]:
root= "/content/drive/My Drive/Kaggle/nexet";
img_dir = "nexet_2017_1/";
anno_file = "train_labels.csv";

In [8]:
dataset_path = root;
images_folder = root + "/" + img_dir;
annotations_path = root + "/annotations/";

In [9]:
if not os.path.isdir(annotations_path):
    os.mkdir(annotations_path)
    
input_images_folder = images_folder;
input_annotations_path = root + "/" + anno_file;

In [10]:
output_dataset_path = root;
output_image_folder = input_images_folder;
output_annotation_folder = annotations_path;

tmp = img_dir.replace("/", "");
output_annotation_file = output_annotation_folder + "/instances_" + tmp + ".json";
output_classes_file = output_annotation_folder + "/classes.txt";

In [11]:
if not os.path.isdir(output_annotation_folder):
    os.mkdir(output_annotation_folder);

In [12]:
df = pd.read_csv(input_annotations_path);
columns = df.columns

In [13]:
delimiter = " ";

In [14]:
list_dict = [];
anno = [];
for i in range(len(df)):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(delimiter);
    for j in range(len(tmp)//5):
        label = tmp[j*5+4];
        if(label not in anno):
            anno.append(label);
    anno = sorted(anno)
    
for i in tqdm(range(len(anno))):
    tmp = {};
    tmp["supercategory"] = "master";
    tmp["id"] = i;
    tmp["name"] = anno[i];
    list_dict.append(tmp);

anno_f = open(output_classes_file, 'w');
for i in range(len(anno)):
    anno_f.write(anno[i] + "\n");
anno_f.close();

100%|██████████| 5/5 [00:00<00:00, 5070.48it/s]


In [15]:
coco_data = {};
coco_data["type"] = "instances";
coco_data["images"] = [];
coco_data["annotations"] = [];
coco_data["categories"] = list_dict;
image_id = 0;
annotation_id = 0;


for i in tqdm(range(len(df))):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(delimiter);
    image_in_path = input_images_folder + "/" + img_name;
    img = cv2.imread(image_in_path, 1);
    h, w, c = img.shape;

    images_tmp = {};
    images_tmp["file_name"] = img_name;
    images_tmp["height"] = h;
    images_tmp["width"] = w;
    images_tmp["id"] = image_id;
    coco_data["images"].append(images_tmp);
    

    for j in range(len(tmp)//5):
        x1 = int(float(tmp[j*5+0]));
        y1 = int(float(tmp[j*5+1]));
        x2 = int(float(tmp[j*5+2]));
        y2 = int(float(tmp[j*5+3]));
        label = tmp[j*5+4];
        annotations_tmp = {};
        annotations_tmp["id"] = annotation_id;
        annotation_id += 1;                                                                 
        annotations_tmp["image_id"] = image_id;
        annotations_tmp["segmentation"] = [];
        annotations_tmp["ignore"] = 0;
        annotations_tmp["area"] = (x2-x1)*(y2-y1);
        annotations_tmp["iscrowd"] = 0;
        annotations_tmp["bbox"] = [x1, y1, x2-x1, y2-y1];
        annotations_tmp["category_id"] = anno.index(label);

        coco_data["annotations"].append(annotations_tmp)
    image_id += 1;

outfile =  open(output_annotation_file, 'w');
json_str = json.dumps(coco_data, indent=4);
outfile.write(json_str);
outfile.close();

100%|██████████| 49281/49281 [6:10:22<00:00,  2.22it/s]


In [1]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

Cloning into 'Monk_Object_Detection'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 6915 (delta 86), reused 65 (delta 33), pack-reused 6740
Receiving objects: 100% (6915/6915), 206.40 MiB | 39.45 MiB/s, done.
Resolving deltas: 100% (2697/2697), done.
Checking out files: 100% (5499/5499), done.


In [73]:
! cd Monk_Object_Detection/3_mxrcnn/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install

In [171]:
import os
import sys
sys.path.append("Monk_Object_Detection/3_mxrcnn/lib/")
sys.path.append("Monk_Object_Detection/3_mxrcnn/lib/mx-rcnn")

In [172]:
from train_base import *

In [168]:
root_dir = "/content/drive/My Drive/Kaggle/";
coco_dir = "nexet";
img_dir = "nexet_2017_1";



In [169]:

set_dataset_params(root_dir=root_dir, 
                   coco_dir=coco_dir, imageset=img_dir);

In [170]:

set_model_params(model_name="vgg16");

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO:root:
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-170-00a154e9a416>", line 2, in <module>
    set_model_params(model_name="vgg16");
  File "Monk_Object_Detection/3_mxrcnn/lib/train_base.py", line 112, in set_model_params
FileNotFoundError: [Errno 2] No such file or directory: 'pretrained'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    re

FileNotFoundError: ignored

In [130]:
set_hyper_params(gpus="0", lr=0.001, lr_decay_epoch="1", epochs=4, batch_size=1);

In [131]:
set_output_params(log_interval=100, save_prefix="model_vgg16");

In [132]:
set_img_preproc_params(img_short_side=600, img_long_side=1000, 
                       mean=(123.68, 116.779, 103.939), std=(1.0, 1.0, 1.0));

In [133]:
initialize_rpn_params();

In [134]:
initialize_rcnn_params();

In [139]:

if os.path.isdir("/content/drive/My Drive/Kaggle/nexet/cache/"):
    os.system("rm -r /content/drive/My Drive/Kaggle/nexet/cache/")

In [136]:
roidb = set_dataset();

INFO:root:loading cache /content/drive/My Drive/Kaggle/nexet/cache/coco_nexet_2017_1_roidb.pkl
INFO:root:coco_nexet_2017_1 num_images 49281
INFO:root:filter roidb: 49281 -> 49281
INFO:root:coco_nexet_2017_1 append flipped images to roidb


In [140]:
sym = set_network();

In [141]:
train(sym, roidb);

INFO:root:max input shape
{'bbox_target': (1, 36, 62, 62),
 'bbox_weight': (1, 36, 62, 62),
 'data': (1, 3, 1000, 1000),
 'gt_boxes': (1, 100, 5),
 'im_info': (1, 3),
 'label': (1, 1, 558, 62)}
INFO:root:max output shape
{'bbox_loss_reshape_output': (1, 128, 24),
 'blockgrad8_output': (1, 128),
 'cls_prob_reshape_output': (1, 128, 6),
 'rpn_bbox_loss_output': (1, 36, 62, 62),
 'rpn_cls_prob_output': (1, 2, 558, 62)}


MXNetError: ignored